1. 数据清洗和处理：对历史数据进行清洗和处理，包括去除异常值、缺失值处理等。此外，还需要将数据按照时间序列的方式进行排序。
2. 时间序列分解：将时间序列数据分解为趋势、季节和随机成分。这可以通过拟合加法模型或乘法模型来实现。其中加法模型假定季节成分与趋势成分之和等于原始数据，而乘法模型假定季节成分与趋势成分的乘积等于原始数据。
3. 模型选择和拟合：选择合适的时间序列模型对趋势、季节和随机成分进行拟合。一般常用的模型包括ARIMA模型、指数平滑模型等。
4. 模型诊断：对拟合好的模型进行诊断，检验其残差是否符合正态分布、是否存在自相关性等。
5. 模型预测：使用已经拟合好的模型进行未来需求量的预测，并计算预测精度。

针对本问题，建议分别采用日、周、月三种时间粒度进行预测，通过比较预测结果的误差，得出不同粒度对预测精度的影响。
我们将以月为粒度对训练数据进行聚合，并提取出需要预测的销售区域、产品、产品品类和产品细品的所有组合。对于每个组合，我们将月销售量作为标签，其他特征包括销售区域、产品、产品品类、产品细品以及月份，我们将这些特征进行独热编码处理。我们将最后 3 个月的数据作为验证集，其余数据作为训练集。

接下来，我们将使用ARIMAX来建立模型并进行预测。

# 一、以天为时间粒度

In [3]:
import pandas as pd
import statsmodels.api as sm
from datetime import datetime, timedelta

train_data = pd.read_csv('data/order_train0.csv')
predict_data = pd.read_csv('data/predict_sku0.csv')
train_data['order_date'] = pd.to_datetime(train_data['order_date'])
train_data = train_data.set_index('order_date')
train_ts = train_data.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])['ord_qty'].resample('D').sum()


In [ ]:

def make_stationary(ts):
    # 一阶差分
    ts_diff = ts.diff().dropna()
    # 进一步差分，直到平稳
    while not sm.tsa.stattools.adfuller(ts_diff)[1] < 0.05:
        ts_diff = ts_diff.diff().dropna()
    return ts_diff

train_ts_diff = train_ts.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code']).apply(make_stationary)
order = (1, 1, 1)
seasonal_order = (1, 0, 1, 12)

In [ ]:

model = sm.tsa.statespace.SARIMAX(train_ts_diff, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
result = model.fit()

# 预测未来三个月的数据
start_date = datetime(2019, 1, 1)
end_date = datetime(2019, 3, 31)
predict_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# 预测每个销售区域、产品、大类和细类的需求量
predict = pd.DataFrame()
for i in range(len(predict_data)):
    # 生成外部变量
    predict_exog = pd.DataFrame(predict_data.iloc[i, :]).T.set_index(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])
    predict_exog.index = pd.MultiIndex.from_tuples(predict_exog.index)
    predict_exog = predict_exog.reindex(index=train_ts_diff.index.union(predict_exog.index), fill_value=0).sort_index()
    predict_exog = predict_exog.loc[predict_dates]

    # 预测未来三个月的需求量
    predict_diff = result.get_forecast(steps=len(predict_dates), exog=predict_exog, dynamic=True)

    # 将预测出的差分值加上训练集最后一天的差分值
    predict_diff_predicted = predict_diff.predicted_mean
    predict_diff_predicted = predict_diff_predicted + train_ts_diff.iloc[-1]

    # 将差分值转换为预测值
    predict_predicted = predict_diff_predicted.cumsum() + train_ts.iloc[-1]

    # 将预测结果保存到DataFrame中
    predict_temp = pd.DataFrame({'sales_region_code': [predict_data.iloc[i, 0]], 'item_code': [predict_data.iloc[i, 1]],
                                 'first_cate_code': [predict_data.iloc[i, 2]], 'second_cate_code': [predict_data.iloc[i, 3]],
                                 '2019年1月预测需求量': predict_predicted.loc['2019-01-01':'2019-01-31'].sum(),
                                 '2019年2月预测需求量': predict_predicted.loc['2019-02-01':'2019-02-28'].sum(),
                                 '2019年3月预测需求量': predict_predicted.loc['2019-03-01':'2019-03-31'].sum()})
    predict = pd.concat([predict, predict_temp], ignore_index=True)

# 将预测结果保存到Excel文件中
predict.to_excel('result1.xlsx', index=False)

# 二、以月为时间粒度

In [ ]:
import pandas as pd
import statsmodels.api as sm
from datetime import datetime, timedelta

train_data = pd.read_csv('data/order_train0.csv')
predict_data = pd.read_csv('data/predict_sku0.csv')

train_data['order_date'] = pd.to_datetime(train_data['order_date'])
train_data = train_data.set_index('order_date')

train_ts = train_data.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])['ord_qty'].resample('M').sum()

def make_stationary(ts):
    # 一阶差分
    ts_diff = ts.diff().dropna()
    # 进一步差分，直到平稳
    while not sm.tsa.stattools.adfuller(ts_diff)[1] < 0.05:
        ts_diff = ts_diff.diff().dropna()
    return ts_diff

train_ts_diff = train_ts.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code']).apply(make_stationary)

order = (1, 1, 1)
seasonal_order = (1, 0, 1, 12)

model = sm.tsa.statespace.SARIMAX(train_ts_diff, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
result = model.fit()

# 预测未来三个月的数据

start_date = datetime(2019, 1, 1)
end_date = datetime(2019, 3, 31)
predict_dates = pd.date_range(start=start_date, end=end_date, freq='M')

# 预测每个销售区域、产品、大类和细类的需求量

predict = pd.DataFrame()
for i in range(len(predict_data)):
    # 生成外部变量
    predict_exog = pd.DataFrame(predict_data.iloc[i, :]).T.set_index(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])
    predict_exog.index = pd.MultiIndex.from_tuples(predict_exog.index)
    predict_exog = predict_exog.reindex(index=train_ts_diff.index.union(predict_exog.index), fill_value=0).sort_index()
    predict_exog = predict_exog.loc[predict_dates]
    # 预测未来三个月的需求量
    predict_diff = result.get_forecast(steps=len(predict_dates), exog=predict_exog, dynamic=True)

    # 将预测出的差分值加上训练集最后一月的差分值
    predict_diff_predicted = predict_diff.predicted_mean
    predict_diff_predicted = predict_diff_predicted + train_ts_diff.iloc[-1]

    # 将差分值转换为预测值
    predict_predicted = predict_diff_predicted.cumsum() + train_ts.iloc[-1]

   # 将预测结果保存到DataFrame中
    predict_temp = pd.DataFrame({'sales_region_code': [predict_data.iloc[i, 0]], 'item_code': [predict_data.iloc[i, 1]],
                                 '2019年1月预测需求量': predict_predicted.loc['2019-01-01':'2019-01-31'].sum(),
                                 '2019年2月预测需求量': predict_predicted.loc['2019-02-01':'2019-02-28'].sum(),
                                 '2019年3月预测需求量': predict_predicted.loc['2019-03-01':'2019-03-31'].sum()})
    predict = pd.concat([predict, predict_temp], ignore_index=True)

# 将预测结果保存到Excel文件中
predict.to_excel('result1.xlsx', index=False)
